In [1]:
%run /Users/hims/sjsu/bigdata_tech-228/project/customer_segmentation/code/data_prep/data_preperation.ipynb

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


23/11/10 04:40:30 WARN Utils: Your hostname, mac.local resolves to a loopback address: 127.0.0.1; using 10.0.0.39 instead (on interface en0)
23/11/10 04:40:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/10 04:40:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/10 04:40:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/11/10 04:40:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/11/10 04:40:31 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|         city|               hours|is_open|  latitude|   longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|{ByAppointmentOnl...|Pns2l4eNsfO8kk83d...|[Doctors, Traditi...|Santa Barbara|                NULL|      0|34.4266787|-119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|{BusinessAcceptsC...|mpf3x-BjTdTEA3yCZ...|[Shipping Centers...|       Affton|{Monday -> 0:0-0:...|      1| 38.551126|  -90.335695|    

In [ ]:
'''
root
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- address: string (nullable = true)
 |-- attributes: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- city: string (nullable = true)
 |-- hours: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- is_open: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fans: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- yelping_since: timestamp (nullable = true)



metric -> frequency, stars
'''

In [ ]:
columns = {
    "user_id" : "user_id", 
    "elites" : "elites", 
    "days_on_platform" : "yelping_since", 
    "categories_freq": "categories", # Map type
    "number_of_unique_business": "business_id",
    "number_of_visits_per_business" : "business_id",
    "first_seen" : "review_date",
    "last_seen" : "review_date"
}

In [ ]:
spark = init_spark()

In [ ]:
def prepare_datasets():
    business_df = process_business_data(spark)
    user_df = process_user_data(spark)
    friends_df = process_friends_data(spark)
    checkin_df = process_checkin_data(spark)
    tip_df = process_tip_data(spark)
    review_df = process_review_data(spark)
    
    review_df.createOrReplaceTempView("review")
    joined_df = review_df.join(business_df, on = ["business_id"]).join(user_df, on = ["user_id"])
    joined_df.createOrReplaceTempView("joined_df")



In [ ]:
def get_customer_agg_value(spark):
    df = spark.sql("""
        select 
            user_id, 
            min(date) as first_seen, 
            max(date) as last_seen, 
            DATEDIFF(max(date), min(date)) as date_diff,
            count(distinct business_id) as different_business_count,
            avg(stars) as avg_rating,
            min(stars) as min_stars,
            max(stars) as max_stars
        from review
        group by user_id 
    """)
    return df

In [ ]:
from pyspark.sql.functions import explode, col, create_map, collect_list

def get_customer_category_counts(joined_df):
    df = joined_df.select("user_id", explode("categories").alias("category")) \
        .groupBy("user_id", "category").count() \
        .groupBy("user_id").agg(collect_list(create_map(col("category"), col("count"))))
    df.show()
    return df

get_customer_category_counts(joinedDf)